# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [24]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json
import os

# Import API key
from api_keys import g_key
from api_keys import geo_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [25]:
path = 'output_data/cities.csv'
city_data_df = pd.read_csv(path, encoding='utf-8')
# city_test = pd.DataFrame('/output_data/cities.csv')
city_data_df.head()



,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kapaa,22.0752,-159.3190,84.18,72,75,16.11,US,1668983435
1,1,rikitea,-23.1203,-134.9692,66.24,90,100,11.74,PF,1668983435
2,2,jurado,7.1053,-77.7641,77.50,92,98,5.46,CO,1668983436
3,3,ampanihy,-24.7000,44.7500,70.12,77,96,11.70,MG,1668983436
4,4,busselton,-33.6500,115.3333,55.58,65,0,13.53,AU,1668983437


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [26]:
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' into  locations 
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df['Humidity'].astype(float)


In [27]:
# Create a weather Heatmap layer
fig = gmaps.figure(center=(0, 0), zoom_level=2.20, layout={'width': '1200px', 'height': '500px'})

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='500px', width='1200px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [28]:
ideal = city_data_df.loc[((city_data_df['Humidity'] < 70) &
                          (city_data_df['Humidity'] > 50) &
                          (city_data_df['Max Temp'] > 65.0) & 
                          (city_data_df['Max Temp'] < 85.0) & 
                          (city_data_df['Cloudiness'] < 30))]
# ideal.count()
ideal =ideal.dropna()
ideal.count()

City_ID       10
City          10
Lat           10
Lng           10
Max Temp      10
Humidity      10
Cloudiness    10
Wind Speed    10
Country       10
Date          10
dtype: int64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [29]:
hotel_df = ideal
hotel_df['Hotel Name'] = ""
hotel_df.reset_index(drop=True)
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
162,162,saint-pierre,-21.3393,55.4781,71.28,64,0,6.91,RE,1668983532,
208,208,najran,17.4924,44.1277,65.05,60,14,1.39,SA,1668983560,
233,233,sur,22.5667,59.5289,75.00,64,0,7.67,OM,1668983575,
293,293,makakilo city,21.3469,-158.0858,82.72,56,20,21.85,US,1668983611,
319,319,gizo,-8.1030,156.8419,84.63,61,27,9.86,SB,1668983625,
334,334,bushehr,28.9684,50.8385,69.89,68,0,11.50,IR,1668983634,
378,378,inhambane,-23.8650,35.3833,69.22,65,18,10.65,MZ,1668983660,
415,415,cabo san lucas,22.8909,-109.9124,81.61,57,0,17.27,MX,1668983167,
489,489,flinders,-34.5833,150.8552,68.05,53,29,15.99,AU,1668983700,
537,537,talara,-4.5772,-81.2719,72.36,68,0,19.57,PE,1668983755,


In [30]:
for index, row in hotel_df.iterrows():
    target_coordinates = f"{row['Lat']}, {row['Lng']}"
    print(target_coordinates)

-21.3393, 55.4781
17.4924, 44.1277
22.5667, 59.5289
21.3469, -158.0858
-8.103, 156.8419
28.9684, 50.8385
-23.865, 35.3833
22.8909, -109.9124
-34.5833, 150.8552
-4.5772, -81.2719


In [31]:
hotel_name = []
hotel_lat = []
hotel_lng = []

print("Starting hotel search")

for index, row in hotel_df.iterrows():
# target_coordinates = [row['Lat'], row["Lng"]]
# geocoordinates
    target_coordinates = f"{row['Lat']},{row['Lng']}"
    target_search = "hotel"
    target_radius = 10000
    target_type = "hotel"

    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    my_request = requests.get(base_url, params=params).json()
    try:
        hotel_name.append(my_request['results'][0]['name'])
        hotel_lat.append(my_request['results'][0]['geometry']['location']['lat'])
        hotel_lng.append(my_request['results'][0]['geometry']['location']['lng'])
        
    except (KeyError, IndexError):
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

print('---------------------')
print('Hotel Search Complete')


    


Starting hotel search
---------------------
Hotel Search Complete


In [32]:
hotel_df['Hotel Name'] = hotel_name
hotel_df['Hotel Lat'] = hotel_lat
hotel_df['Hotel Lng'] = hotel_lng


In [33]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [34]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(0, 0), zoom_level=2.20, layout={'width': '1200px', 'height': '500px'})
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='500px', width='1200px'))